# ISS Visibility Notebook

This notebook will demonstrate a few powerful data minig tools to gather the location of the ISS and the weather conditions at the corresponding coordinates to check cloud conditions as that location. 
This data will be formatted into a PANDAS data frame then exported to a CSV file that could be used by other applications. 

## Using API to get  ISS data from the web

This section will perform a basic API call and confirm that everything is working. First we will check our request reponse is valid. This data will be pulled from the open notify api, to learn more about this API check here http://open-notify.org/Open-Notify-API/

In [1]:
import requests
# using the requests library we will request data from open notify.org
response = requests.get("http://api.open-notify.org/iss-now.json")
print(response.status_code)

200


We will then check to see what data is recieved from a request.

In [2]:
import time
import pandas as pd
import json

#test the output
response = requests.get("http://api.open-notify.org/iss-now.json")
print(response.json())

{'timestamp': 1645203834, 'message': 'success', 'iss_position': {'latitude': '-51.3401', 'longitude': '60.5445'}}


## Building our dataset using two Data Sources

This section will build our PANDAS data frame by requesting data every 10 minutes from open notify, then using the location data to pull data from the open weather map API (https://openweathermap.org/) and record both. The last step will be to save this data as a .CSV file that can be shared with other developers.

This simple example will only collect 10 rows of data however it would be easy to run this script as long as desired and to collect data at any interval desired. 

In [3]:
ISS_data = pd.DataFrame(columns=["Longitude", "Latitude", "Timestamp", "Visibility"])

for i in range(0,10):
    #get the ISS location data
    response = requests.get("http://api.open-notify.org/iss-now.json")
    json_response = response.json()
    longitude = json_response['iss_position']['longitude']
    latitude = json_response['iss_position']['latitude']
    timestamp = json_response['timestamp']
    
    # get the the weather data
    
    api_key = "37fa799589f4045eb619e2a8d359f95a"
    url = "https://api.openweathermap.org/data/2.5/onecall?lat=%s&lon=%s&appid=%s&units=metric" % (latitude, longitude, api_key)
    weather = requests.get(url)
    weather_data = json.loads(weather.text)
    current = weather_data["current"]["clouds"]
    
    # we will use a bool value to represent visibility, if it is more than 20% cloudy we will set this false.
    
    if current >20:
        vis = False
    else:
        vis = True
        
    

    row_vals = {"Longitude":longitude, "Latitude":latitude, "Timestamp": timestamp, "Visibility": vis}

    ISS_data = ISS_data.append(row_vals, ignore_index = True)
    
   
    time.sleep(600)


### View the data

In [4]:
ISS_data.head(10)

,Longitude,Latitude,Timestamp,Visibility
0,60.5445,-51.3401,1645203834,False
1,113.1839,-40.7139,1645204435,True
2,142.5091,-13.6495,1645205037,False
3,164.6541,16.5408,1645205638,True
4,-164.2216,42.8491,1645206239,True
5,-109.5427,50.7175,1645206841,False
6,-65.7125,31.5722,1645207442,False
7,-41.0122,2.3697,1645208043,False
8,-17.5365,-27.3159,1645208644,True
9,21.8665,-49.0991,1645209245,False


### Export the data

In [5]:
ISS_data.to_csv('ISS_visibility', index=False)

### Confirm the export was successful

In [6]:
export_test = pd.read_csv('ISS_visibility')

export_test.head(10)

,Longitude,Latitude,Timestamp,Visibility
0,60.5445,-51.3401,1645203834,False
1,113.1839,-40.7139,1645204435,True
2,142.5091,-13.6495,1645205037,False
3,164.6541,16.5408,1645205638,True
4,-164.2216,42.8491,1645206239,True
5,-109.5427,50.7175,1645206841,False
6,-65.7125,31.5722,1645207442,False
7,-41.0122,2.3697,1645208043,False
8,-17.5365,-27.3159,1645208644,True
9,21.8665,-49.0991,1645209245,False


Now we can see the data was correctly saved to a .CSV file and can be loaded back into Python for future use!

Thank you for following alog.